# Stringency Index

School closures; Workplace closures; Cancellation of public events; Restrictions on public gatherings; Closures of public transport; Stay-at-home requirements; Public information campaigns; Restrictions on internal movements; and International travel controls.

In [ ]:
url = 'https://github.com/OxCGRT/covid-policy-tracker/raw/master/data/timeseries/OxCGRT_timeseries_all.xlsx'
stringency_index = (
    pd.read_excel(url, sheet_name=0)
      .query('jurisdiction == "NAT_TOTAL"')
      .drop(columns=['region_code', 'region_name','jurisdiction'])
      .dropna(axis='columns',how='all')
)
stringency_index.columns = stringency_index.columns[:2].tolist() + [pd.to_datetime(col, format='%d%b%Y').date() for col in stringency_index.columns[2:]]

stringency_index_ts = (
    stringency_index.melt(id_vars=['country_code', 'country_name'], var_name='date', value_name='value')
                    .assign(date=lambda dd: pd.to_datetime(dd.date))
                    .sort_values(by=['country_name','date'])
                    .query('"2020-02-01" <= date <= "2022-12-31"')
                    .merge(country_to_iso, left_on='country_code', right_on='iso3')
                    .drop(columns=['country_name','country_code'])
                    .reset_index(drop=True)
                    .set_index(['country','date'])
)

In [ ]:
covid_new_cases_ts = (
    covid_df.loc[:,['date','iso3','new_cases']]
            .query('"2020-02-01" <= date <= "2022-12-31"')
            .assign(date=lambda dd: pd.to_datetime(dd.date))
            .reset_index(drop=True)
            .merge(country_to_iso, left_on='iso3', right_on='iso3')
            .set_index(['country','date'])
            .merge(stringency_index_ts,left_index=True, right_index=True)
            .drop(columns=['value'])
)

In [ ]:
from datetime import datetime

# Define the start and end dates
start_date = datetime(2020, 2, 1)
end_date = datetime(2022, 12, 31)

# Calculate the difference in days
num_days = (end_date - start_date).days

print("Number of days between the dates:", num_days)

In [ ]:
import openpyxl

#stringency_index_ts.index.get_level_values('country').unique()

w = 1000
out_dir = '../resultsidx'
for country in tqdm(['United States of America','Italy','Canada','Germany','United Kingdom','France'], desc='Processing Countries', leave=False):
        covid_ts = covid_new_cases_ts.loc[country]['new_cases']
        stringency_ts = stringency_index_ts.loc[country]['value']
        sdc = SDCAnalysis(ts1=stringency_ts,
                          ts2=covid_ts,
                          fragment_size=w,
                          method='spearman',
                          max_lag=0,
                          min_lag=-14)
        sdc.to_excel(f"{out_dir}/tables/SDC_country_groups/sdc_{w}_{country}_stringencyindex.xlsx")
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            sdc.combi_plot(dpi=120,
                           figsize=(7, 7),
                           title=f"SDC with s={w}: \nCOVID19 daily confirmed cases in" +
                                 f" {country} ~ 'Stringency Index'",
                           xlabel='Stringency Index',
                           ylabel=f"Confirmed COVID19 cases in {country}",
                           alpha=0.05,
                           max_lag=0,
                           min_lag=-14,
                           max_r=1,
                           wspace=.35,
                           hspace=.35)
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{country}_stringencyindex.png")
        plt.savefig(f"{out_dir}/figures/SDC_country_groups/sdc_{w}_{country}_stringencyindex.pdf")